In [1]:
from pathlib import Path
import pandas as pd
from openpyxl import load_workbook
import os
import numpy as np
import pickle

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
PATH_FILES_ONE_DRIVE = Path(r'C:\Users\h-pgy\one_drive_prefs\OneDrive - Default Directory\Shared Documents\Estruturação do PDM 2021-2024\Elaboração PDM Versão Final')

In [4]:
class PlanilhaNotFound(ValueError):
    '''Raised quando nao consegue achar planilha para a meta'''

In [5]:
def listar_metas_ok():
    
    controle = pd.read_excel(PATH_FILES_ONE_DRIVE/'Controle das Devolutivas.xlsx')
    
    filtro_preench = controle['Aba Orçamento preenchida corretamente?'].str.lower().str.strip()=='sim'
    filtro_analis = controle['Aba orçamento já analisada?'].str.lower().str.strip()=='sim'
    
    for m in controle[~(filtro_preench&filtro_analis)]['Meta'].unique():
        print(f'Meta {m} não está ok no controle')
    
    return controle[filtro_preench&filtro_analis]['Meta'].values

In [6]:
def get_planilhas(folder_original):
        
        fpaths =  [os.path.join(folder_original, file) for file in 
                    os.listdir(folder_original) if 'xls' 
                    in file.split('.')[-1]]
        
        return fpaths

In [7]:
def read_workbook(file):

    wb = load_workbook(file, read_only=True, data_only=True)

    return wb

In [8]:
def match_planilha(wb, num_meta):
        
        sheet = wb['Ficha técnica Meta ']
        
        num_meta_planilha = sheet['C5'].value
        
        if str(num_meta_planilha) == str(num_meta):
            return True

In [9]:
def find_planilha(files, num_meta):
    
    for file in files:
        
        wb = read_workbook(file)
        
        if match_planilha(wb, num_meta):
            secretaria = wb['Ficha técnica Meta ']['C4'].value
            files.remove(file) # remove para dar uma otimizada nas próximas buscas
            return file, secretaria
    else:
        raise PlanilhaNotFound(f'Planilha não encontrada para a meta {num_meta}')

In [10]:
def planilhao(lista_ok = None, path_xls = None, lista_dfs = None):
    
    if path_xls is None:
        path_xls = PATH_FILES_ONE_DRIVE/'Fichas Metas\Devolutiva 11-jun'
        
    if lista_ok is None:
        lista_ok = listar_metas_ok()
        
    if lista_dfs is None:
        lista_dfs = []
    
    f_paths = get_planilhas(path_xls)
        
    for meta in lista_ok:
        print(f'Puxando dados meta {meta}')
        try:
            file, secretaria = find_planilha(f_paths, meta)
        except PlanilhaNotFound as e:
            print(e)
            continue
        df = pd.read_excel(file, sheet_name = 'Orçamento', thousands = '.',
                          dtype = {'Código Ação' : str})
        df['meta'] = meta
        df['secretaria'] = secretaria
        lista_dfs.append(df)
    
    return pd.concat(lista_dfs)
    

In [11]:
lista_dfs = []
df = planilhao(lista_dfs = lista_dfs)

Meta 1 não está ok no controle
Meta 53 não está ok no controle
Meta 54 não está ok no controle
Meta 69 não está ok no controle
Meta SMSU Nova 1 não está ok no controle
Meta SGM.Nova1 não está ok no controle
Meta SGM.Nova2 não está ok no controle
Meta nan não está ok no controle
Puxando dados meta 2
Puxando dados meta 3.1
Puxando dados meta 4
Puxando dados meta 5.1
Puxando dados meta 6
Puxando dados meta 7
Puxando dados meta 8
Puxando dados meta 9
Puxando dados meta 10
Puxando dados meta 11
Puxando dados meta 12
Puxando dados meta 13
Puxando dados meta 14
Puxando dados meta 15
Puxando dados meta 16
Puxando dados meta 17
Puxando dados meta 18
Puxando dados meta 19
Puxando dados meta 20
Puxando dados meta 21
Puxando dados meta 22
Puxando dados meta 23
Puxando dados meta 24
Puxando dados meta 25
Puxando dados meta 26
Puxando dados meta 27
Puxando dados meta 28
Puxando dados meta 29
Puxando dados meta 30
Puxando dados meta 31
Puxando dados meta 32
Puxando dados meta 33
Puxando dados meta 34

In [12]:
with open('orcamento.pi', 'wb') as f:
    pickle.dump(df, f)

In [13]:
with open('orcamento.pi', 'rb') as f:
    df = pickle.load(f)

In [14]:
def dropar_unnamed(df):
    
    unnameds = [col for col in df if
               col.startswith('Unnamed: ')]
    
    df.drop(unnameds, axis = 1, inplace = True)
    for col in df:
        if col.startswith('Unnamed: '):
            df.drop(col, axis = 1, )

In [15]:
dropar_unnamed(df)

In [16]:
sem_na = df.dropna(how='all', subset = [col for col in df.columns if col not in ('meta', 'secretaria')]).copy()

In [17]:
def pegar_objetivos(df):
    
    objetivos = pd.read_excel(PATH_FILES_ONE_DRIVE/'Metas com Objetivos atualizados 18-06-21.xlsx')
    
    merged = df.merge(objetivos, how = 'left', 
                      right_on = 'Nº meta PdM ',  left_on = 'meta').copy()
    
    merged.drop(['Redação da Meta PdM ','Nº meta PdM '],
               axis = 1, inplace = True)
    
    return merged

In [18]:
sem_na = pegar_objetivos(sem_na)

In [19]:
def arrumar_meta(item):
    
    item = str(item)
    
    antes, depois = item.split('.')
    
    if int(depois) > 0:
        return f'{antes}.{depois}'
    else:
        return antes

In [20]:
sem_na['meta'] = sem_na['meta'].apply(arrumar_meta)

In [21]:
def add_ponto_acao(item):
    
    item = str(item)
    
    if item == 'nan':
        return 'XX.XX.XXXX.XX'
    else:
        return f'{item[:2]}.{item[2:4]}.{item[4:8]}.{item[8:10]}'

In [22]:
def gerar_colunas_dotacao(df):
    
    
    df = df.copy()
    df['Código Ação'] = df['Código Ação'].apply(add_ponto_acao)
    df['Unidade Orçamentária'] = df['Código Ação'].apply(lambda x: '.'.join(x.split('.')[:2]))
    df['Ação'] = df['Código Ação'].apply(lambda x: x.split('.')[2])
    df['Fonte'] = df['Código Ação'].apply(lambda x: x.split('.')[3])
    
    return df

In [23]:
sem_na = gerar_colunas_dotacao(sem_na)

In [24]:
ordem_cols = [
    'Objetivo Estratégico Atualizado',
    'secretaria',
    'meta',
    'IN',
    'Item',
    'Custeio/\nInvestimento',
    'Código Ação',
    'Unidade Orçamentária',
    'Ação',
    'Fonte',
    'classif.',
    'V. Atualizado',
    'V. Congelado',
    'V. Disponível',
    'Atualizado (Disponibilizado PDM)',
    'Congelado (Dispon. PDM)',
    'Disponível (Dispon. PDM)',
    'Custo 2021',
    'Custo 2022',
    'Custo 2023',
    'Custo 2024',
    'Custo TOTAL',
     
     ]

In [25]:
sem_na = sem_na[ordem_cols]

In [26]:
sem_na.to_excel('planilhao_orcamento_final.xlsx', index = False)

In [27]:
colunas_numericas = [
'V. Atualizado',
'V. Congelado',                       
'V. Disponível',                        
'Atualizado (Disponibilizado PDM)',
'Congelado (Dispon. PDM)',    
'Disponível (Dispon. PDM)',             
'Custo 2021',            
'Custo 2022',                          
'Custo 2023',                          
'Custo 2024',                          
'Custo TOTAL']

In [28]:
df[df.eq('Não se aplica').any(1)]

,IN,Item,Custeio/\nInvestimento,Código Ação,classif.,V. Atualizado,V. Congelado,V. Disponível,Atualizado (Disponibilizado PDM),Congelado (Dispon. PDM),Disponível (Dispon. PDM),Custo 2021,Custo 2022,Custo 2023,Custo 2024,Custo TOTAL,meta,secretaria,Registro da alteração,Suplementações previstas\n(Disponibilizado PDM),Observações sobre as suplementações previstas
0,a,1,Não se aplica,Não se aplica,execução,NaN,NaN,NaN,Não se aplica,Não se aplica,NaN,Não se aplica,Não se aplica,Não se aplica,Não se aplica,0.0,20.0,SME,NaN,NaN,NaN
3,c,1,Não se aplica,Não se aplica,execução,NaN,NaN,NaN,Não se aplica,Não se aplica,NaN,0.0,0.0,0.0,0.0,0.0,21.0,SME,NaN,NaN,NaN


In [29]:
def solve_numero(item):
    
    if item == 'Não se aplica':
        return np.nan
    if type(item) in (float, int):
        return item
    elif type(item) is str:
        try:
            float(item)
        except ValueError:
            try:
                item = item.replace('.', '')
                item = item.replace(',', '.')
                return float(item)
            except ValueError:
                print(f'Item {item} fora do padrão')
                return np.nan
    else:
        print(type(item))

In [30]:
numerico = sem_na.copy()
for col in colunas_numericas:
    numerico[col] = numerico[col].apply(solve_numero)

Item 0.00
0.00 fora do padrão


In [31]:
numerico.dtypes

Objetivo Estratégico Atualizado      object
secretaria                           object
meta                                 object
IN                                   object
Item                                 object
Custeio/\nInvestimento               object
Código Ação                          object
Unidade Orçamentária                 object
Ação                                 object
Fonte                                object
classif.                             object
V. Atualizado                       float64
V. Congelado                        float64
V. Disponível                       float64
Atualizado (Disponibilizado PDM)    float64
Congelado (Dispon. PDM)             float64
Disponível (Dispon. PDM)            float64
Custo 2021                          float64
Custo 2022                          float64
Custo 2023                          float64
Custo 2024                          float64
Custo TOTAL                         float64
dtype: object

In [32]:
n_pode_vazio = [
    'Atualizado (Disponibilizado PDM)',
    'Congelado (Dispon. PDM)',
    'Disponível (Dispon. PDM)',
    'Custo 2021',
    'Custo 2022',
    'Custo 2022',
    'Custo 2023',
    'Custo 2024'
]

In [33]:
for col in n_pode_vazio:
    soma = numerico[col].isnull().sum()
    if  soma > 0:
        print(col, soma)

Atualizado (Disponibilizado PDM) 7
Congelado (Dispon. PDM) 10
Disponível (Dispon. PDM) 16
Custo 2021 14
Custo 2022 12
Custo 2022 12
Custo 2023 11
Custo 2024 11


In [34]:
dropar_categoricas = (numerico['classif.'].isnull()|numerico['Custeio/\nInvestimento'].isnull())

In [35]:
numerico[dropar_categoricas]

,Objetivo Estratégico Atualizado,secretaria,meta,IN,Item,Custeio/\nInvestimento,Código Ação,Unidade Orçamentária,Ação,Fonte,classif.,V. Atualizado,V. Congelado,V. Disponível,Atualizado (Disponibilizado PDM),Congelado (Dispon. PDM),Disponível (Dispon. PDM),Custo 2021,Custo 2022,Custo 2023,Custo 2024,Custo TOTAL
2,Garantir à população atendimento integral em s...,SMS,2,NaN,NaN,NaN,XX.XX.XXXX.XX,XX.XX,XXXX,XX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
6,Garantir à população atendimento integral em s...,SMS,4,NaN,NaN,NaN,XX.XX.XXXX.XX,XX.XX,XXXX,XX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
441,Promover a transparência e a integridade dos ó...,CGM,73,NaN,NaN,NaN,XX.XX.XXXX.XX,XX.XX,XXXX,XX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150000.0


In [36]:
limpo = numerico[~dropar_categoricas].copy()

In [37]:
dropar_numericas = (limpo['Atualizado (Disponibilizado PDM)'].isnull()|
                    limpo['Congelado (Dispon. PDM)'].isnull()|
                    limpo['Disponível (Dispon. PDM)'].isnull()|
                    limpo['Custo 2021'].isnull()|
                    limpo['Custo 2022'].isnull()|
                    limpo['Custo 2023'].isnull()|
                    limpo['Custo 2024'].isnull())

In [38]:
dropar_numericas.sum()

28

In [39]:
limpo[dropar_numericas]

,Objetivo Estratégico Atualizado,secretaria,meta,IN,Item,Custeio/\nInvestimento,Código Ação,Unidade Orçamentária,Ação,Fonte,classif.,V. Atualizado,V. Congelado,V. Disponível,Atualizado (Disponibilizado PDM),Congelado (Dispon. PDM),Disponível (Dispon. PDM),Custo 2021,Custo 2022,Custo 2023,Custo 2024,Custo TOTAL
82,Garantir à toda população em idade escolar o a...,SME,20,a,1,Não se aplica,Nã.o .se a.pl,Nã.o,se a,pl,execução,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
86,Garantir à toda população em idade escolar o a...,SME,20,d,2,Custeio,16.10.2831.00,16.10,2831,00,execução,5.359134e+07,0.0,5.359134e+07,0.00,0.000000e+00,NaN,0.000000e+00,5.040000e+03,5.292000e+03,5.556600e+03,1.588860e+04
87,Garantir à toda população em idade escolar o a...,SME,20,d,3,Custeio,16.10.2180.00,16.10,2180,00,execução,1.435809e+07,0.0,1.435809e+07,0.00,0.000000e+00,NaN,0.000000e+00,6.788250e+03,7.127660e+03,7.484050e+03,2.139996e+04
91,Garantir à toda população em idade escolar o a...,SME,21,c,1,Não se aplica,Nã.o .se a.pl,Nã.o,se a,pl,execução,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
105,Garantir à toda população em idade escolar o a...,SME,21,e,1,Custeio,16.10.2820.00,16.10,2820,00,execução,9.507020e+05,0.0,9.507020e+05,0.00,0.000000e+00,NaN,0.000000e+00,1.050000e+06,0.000000e+00,1.050000e+06,2.100000e+06
116,Democratizar o acesso a políticas públicas de ...,SEME,26,b,2,Custeio,19.10.2397.00,19.10,2397,00,execução,2.501000e+06,1000.0,2.500000e+06,0.00,0.000000e+00,NaN,0.000000e+00,5.550000e+05,1.110000e+06,1.665000e+06,3.330000e+06
117,Democratizar o acesso a políticas públicas de ...,SEME,26,c,3,Custeio,19.10.2397.00,19.10,2397,00,execução,2.501000e+06,1000.0,2.500000e+06,0.00,0.000000e+00,NaN,0.000000e+00,1.105000e+06,1.215500e+06,1.337050e+06,3.657550e+06
129,"Ampliar a resiliência da cidade às chuvas, red...",SIURB,30,A,3,Investimento,86.22.5013.03,86.22,5013,03,execução,1.895025e+08,189502484.0,0.000000e+00,NaN,1.895025e+08,0.00,3.906923e+08,2.813403e+08,1.920251e+08,1.111058e+08,9.751634e+08
137,"Ampliar a resiliência da cidade às chuvas, red...",SMSUB,31,A,1,Custeio,xx..x.x.xx.xx,xx.,x,x,contrapartida,NaN,NaN,NaN,8599612.25,0.000000e+00,8599612.25,NaN,NaN,NaN,NaN,0.000000e+00
141,Garantir a qualidade e segurança das vias públ...,SMSUB,33,A,3,Custeio,12.10.2340.00,12.10,2340,00,execução,1.113350e+08,0.0,1.113350e+08,0.00,0.000000e+00,NaN,1.624554e+08,1.787009e+08,1.965710e+08,2.162281e+08,7.539555e+08


In [40]:
# a principio nao vamos limpar as numericas
#limpo = limpo[~dropar_numericas].copy()

In [41]:
limpo[limpo['Objetivo Estratégico Atualizado'].isnull()]

,Objetivo Estratégico Atualizado,secretaria,meta,IN,Item,Custeio/\nInvestimento,Código Ação,Unidade Orçamentária,Ação,Fonte,classif.,V. Atualizado,V. Congelado,V. Disponível,Atualizado (Disponibilizado PDM),Congelado (Dispon. PDM),Disponível (Dispon. PDM),Custo 2021,Custo 2022,Custo 2023,Custo 2024,Custo TOTAL
486,NaN,SMS,3.2,C,1,investimento,84.11.9204.01,84.11,9204,01,execução,223111204.0,0.0,223111204.0,0.0,0.0,0.0,2000000.0,28000000.0,5000000.0,5000000.0,40000000.0
487,NaN,SMS,3.2,C,2,investimento,84.11.9204.01,84.11,9204,01,execução,NaN,0.0,223111204.0,0.0,0.0,0.0,2000000.0,28000000.0,5000000.0,5000000.0,40000000.0
488,NaN,SMS,3.2,C,3,investimento,84.11.9204.00,84.11,9204,00,execução,80002000.0,2000.0,80000000.0,0.0,0.0,0.0,2000000.0,28000000.0,5000000.0,5000000.0,40000000.0
489,NaN,SMS,3.2,C,4,investimento,84.11.9204.01,84.11,9204,01,execução,223111204.0,0.0,223111204.0,0.0,0.0,0.0,15000000.0,15000000.0,15000000.0,15000000.0,60000000.0
490,NaN,SMS,3.2,A,1,investimento,84.11.9204.00,84.11,9204,00,execução,80002000.0,2000.0,80000000.0,0.0,0.0,0.0,40000000.0,100000000.0,60000000.0,0.0,200000000.0
491,NaN,SMS,3.2,B,1,investimento,84.11.9204.00,84.11,9204,00,execução,80002000.0,2000.0,80000000.0,NaN,NaN,NaN,2000000.0,4000000.0,2000000.0,0.0,8000000.0


In [42]:
limpo.to_excel('orcamento_limpo_final.xlsx')